In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
http://news.metrotvnews.com/index/2018/07/30/30

## GET INDEKS LINK

In [14]:
def getIndeksLink(links, page, offset, cat_link, category, date=datetime.strftime(datetime.today(), '%Y/%m/%d')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : YYYY-mm-dd
    """
    print("page ", page)
    url = "http://"+cat_link+".metrotvnews.com/index/"+date+"/"+ str(offset)
    print(url)
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    contentDiv = soup.find('div', class_="style_06").findAll('h2')
    contentDiv
    for post in contentDiv:
        link = [post.find('a',href=True)['href']]
        links.append(link)
            
    el_page = soup.find('div', class_="extrapadding")
    if el_page:
        a_page = el_page.findAll('div', class_='bu fr')[-1].find('a')
        max_page = int(a_page['data-ci-pagination-page'].replace('\n', '').strip(' '))
        
        if page < max_page:
            links = getIndeksLink(links, page+1, offset+30, cat_link, category, date)    
        
    return links

## GET DETAIL BERITA

In [33]:
def getDetailBerita(url):
    all_articles = []
    for link in links:
        articles = {}
        #link
        url = link[0]
        response = requests.get(url)
        html = response.text

        #Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")

        #extract subcategory from breadcrumb
        bc = soup.find('div', class_="breadcrumbs")
        if not bc:
            continue
        cat = bc.findAll('a')[-2].text
        sub = bc.findAll('a')[-1].text

        #articles['id'] = int(soup.find("meta", attrs={'name':'articleid'})['content'])

        #category
        #category
        articles['category'] = cat
        articles['subcategory'] = sub
        
        articles['url'] = url

        article = soup.find('div', class_="tru")

        #extract date
        pubdate_author = soup.find("div",class_="reg").text
        pubdate_author_split = pubdate_author.split(' \xa0\xa0 • \xa0\xa0 ')
        pubdate = pubdate_author_split[1]
        pubdate = pubdate.strip(' \t\n\r')
        pubdate = pubdate.replace(' WIB','')
        articles['pubdate'] = datetime.strftime(datetime.strptime(pubdate, "%A, %d %b %Y %H:%M"), "%Y-%m-%d %H:%M:%S")

        #extract author
        author = pubdate_author_split[0]
        articles['author'] = author

        #extract title
        articles['title'] = soup.find('meta', attrs={"property":"og:title"})['content']
        if ("foto" in sub.lower()) or  "video" in sub.lower(): 
            continue
            
        #source
        articles['source'] = 'metrotvnews'

        #extract comments count
        #articles['comments'] = int(soup.find('a', class_="komentar").find('span').text.replace('Komentar', '').strip(' \t\n\r'))
        #articles['comment'] = print("?")

        #extract tags
        tags = soup.find('div', class_="line").findAll('a', class_="tag")
        articles['tags'] = ','.join([x.text for x in tags])

        #extract images
        articles['images'] = soup.find('img', class_="pic")['src']

        #extract detail
        detail = soup.find('div', class_="tru")

        #hapus link sisip
        for link in detail.findAll('div', class_="related"):
            link.decompose()

        #hapus video sisip
        for tag in detail.findAll('iframe', class_="embedv"):
            tag.decompose()

        #hapus all setelah clear fix
        #for det in detail.find('div', class_="wfull fl rl"):
        #    det.decompose()

        #hapus all script
        for script in detail.findAll('script'):
            script.decompose()

        #extract content
        detail = BeautifulSoup(detail.decode_contents().replace('<br/>', ' '), "html5lib")
        content = re.sub(r'\n|\t|\b|\r','',detail.text)
        articles['content'] = content.strip(' ')
        #print('memasukkan berita id ', articles['id'])
        #all_articles.append(articles)
    return articles

In [18]:
links = getIndeksLink([], 1, 0, 'news', 'news')

page  1
http://news.metrotvnews.com/index/2018/07/31/0
page  2
http://news.metrotvnews.com/index/2018/07/31/30


In [19]:
links

[['http://news.metrotvnews.com/metro/VNx7ODJK-lrt-jakarta-dibuka-10-agustus'],
 ['http://news.metrotvnews.com/metro/zNAwE18k-polri-tagih-pergub-ganjil-genap-ke-pemprov-dki'],
 ['http://news.metrotvnews.com/politik/4KZ4B9gb-sandiaga-uno-mundur-sebagai-ketua-pemenangan-pemilu-gerindra'],
 ['http://news.metrotvnews.com/peristiwa/ybDOGgvN-polri-minta-masyarakat-membantu-pergerakan-atlet'],
 ['http://news.metrotvnews.com/politik/aNrDzR2k-koalisi-pendukung-prabowo-diklaim-solid'],
 ['http://news.metrotvnews.com/metro/yKX93Z7N-sandi-sebut-sudah-tandatangani-pergub-ganjil-genap'],
 ['http://news.metrotvnews.com/peristiwa/4KZ4BAYb-guru-wajib-beri-tugas-siswa-yang-libur'],
 ['http://news.metrotvnews.com/politik/8N0V4Rrk-pbb-kpu-capai-kesepakatan-lewat-mediasi-di-bawaslu'],
 ['http://news.metrotvnews.com/hukum/GKdW8eAk-pangeran-qatar-dapat-pengamanan-ektra-di-asian-games'],
 ['http://news.metrotvnews.com/hukum/nN95p33N-jad-dibekukan'],
 ['http://news.metrotvnews.com/metro/8korj5rb-ganjil-genap-di

In [34]:
details = [getDetailBerita(links[:2])]
details

[{'author': ' Yogi Bayu Aji',
  'category': 'News',
  'content': 'Jakarta: Tim gabungan berhasil mengevakuasi 543 pendaki yang terjebak di Gunung Rinjani, Nusa Tenggara Barat (NTB). Para pendaki yang berhasil dievakuasi terdiri dari 189 pendaki warga negara asing, 173 warga negara Indonesia, 150 orang porter, dan 31 orang guide.  Sebelum mengevakuasi, tim gabungan yang berjumlah 122 personel berhasil melakukan kontak dengan para wisatawan pendaki Gunung Rinjani. Mereka kemudian dievakuasi lantaran sejumlah jalur pendakian tertutup material longsor pascagempa.     Hingga saat ini, tim gabungan masih terus mengevakuasi para pendaki wisatawan lokal dan asing yang terperangkap akibat gempa bumi 6,4 SR yang terjadi beberapa waktu yang lalu.\xa0  "Tim gabungan evakuasi pendaki Gunung Rinjani masih berusaha keras menyelamatkan pendaki gunung yang masih berada di atas Gunung Rinjani," kata Kabidpenum Puspen TNI Kolonel Sus Taibur Rahman dalam keterangan resmi yang diterima Medcom.id, Jakarta, 

In [35]:
df = pd.DataFrame(details)

In [36]:
df

,author,category,content,images,pubdate,source,subcategory,tags,title,url
0,Yogi Bayu Aji,News,Jakarta: Tim gabungan berhasil mengevakuasi 54...,http://cdn.metrotvnews.com/dynamic/content/201...,2018-07-31 02:10:00,metrotvnews,Daerah,"gempa,gempa bumi",Tim Gabungan Evakuasi 543 Pendaki Rinjani,http://news.metrotvnews.com/daerah/5b2VPrrb-ti...
